In [1]:
# https://stackoverflow.com/questions/39299838/how-do-i-import-module-in-jupyter-notebook-directory-into-notebooks-in-lower-dir
# https://stackoverflow.com/questions/68572852/import-local-modules-in-jupyter-notebook
import os
import sys
# os.path.split(os.getcwd())[0]
sys.path.append("/gpfs/home/dhuang/thesis/WavCaps/retrieval/models")
sys.path.append("/gpfs/home/dhuang/thesis/WavCaps/retrieval/tools")
for nb_dir in sys.path:
    print(nb_dir)
    # sys.path.append(nb_dir)


/gpfs/home/dhuang/thesis/WavCaps
/home/dhuang/.conda/envs/fs/lib/python310.zip
/home/dhuang/.conda/envs/fs/lib/python3.10
/home/dhuang/.conda/envs/fs/lib/python3.10/lib-dynload

/home/dhuang/.conda/envs/fs/lib/python3.10/site-packages
/home/dhuang/.conda/envs/fs/lib/python3.10/site-packages/huggingface_hub-0.14.1-py3.8.egg
/home/dhuang/.conda/envs/fs/lib/python3.10/site-packages/mpmath-1.2.1-py3.10.egg
/home/dhuang/.conda/envs/fs/lib/python3.10/site-packages/sentencepiece-0.1.95-py3.10-linux-x86_64.egg
/gpfs/home/dhuang/thesis/WavCaps/retrieval/models
/gpfs/home/dhuang/thesis/WavCaps/retrieval/tools


In [1]:
import torch
import torch.nn as nn
from models.audio_encoder import AudioEncoder
from models.text_encoder import TextEncoder
import torch.nn.functional as F
import copy
from tools.losses import AudioTextContrastiveLoss, NTXent
from tools.utils import remove_grad
import ruamel.yaml as yaml
import librosa
import random
import numpy as np


INFO:numexpr.utils:Note: NumExpr detected 56 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [2]:
config_file_path = './settings/vamp.yaml'

with open(config_file_path, "r") as f:
        config = yaml.safe_load(f)

In [3]:
# Load audio signal file
wav_file_path = '../../dac/audio_samples/at2_cvt.wav'
waveform, _ = librosa.load(wav_file_path, sr=config["audio_args"]["sr"], mono=True)
print('waveform shape before crop: ', waveform.shape)
if config["audio_args"]["max_length"] != 0:
            # if audio length is longer than max_length, we random crop it
            max_length = config["audio_args"]["max_length"] * config["audio_args"]["sr"]
            if waveform.shape[-1] > max_length:
                max_start = waveform.shape[-1] - max_length
                start = random.randint(0, max_start)
                waveform = waveform[start: start + max_length]
                
print('waveform shape: ', waveform.shape)
waveform_tensor = torch.tensor(waveform[None, :])
print('waveform_tensor shape: ', waveform_tensor.shape)


waveform shape before crop:  (661500,)
waveform shape:  (441000,)
waveform_tensor shape:  torch.Size([1, 441000])


In [4]:
batch_size = 5
batch_waveform_tensor = waveform_tensor.repeat(batch_size, 1)
print(batch_waveform_tensor.shape)

torch.Size([5, 441000])


In [5]:
audio_encoder = AudioEncoder(config)
# settings for projection layers
embed_size = config["embed_size"]
audio_width = audio_encoder.audio_width

loading model hugggof/vampnet-models:vampnet-9codebook-linear-sched-best from the huggingface hub.


In [6]:
audio_feats = audio_encoder(batch_waveform_tensor)
# audio_embeds = F.normalize(self.audio_proj(audio_feats), dim=-1)
print('audio_encoded.shape: ', audio_feats.shape)
audio_feats

audio_encoded.shape:  torch.Size([1, 1280, 861])


tensor([[[ 0.1130,  0.1130,  0.1130,  ...,  0.1130,  0.1130,  0.1130],
         [-0.1961, -0.1961, -0.1961,  ..., -0.1961, -0.1961, -0.1961],
         [-0.1603, -0.1603, -0.1603,  ..., -0.1603, -0.1603, -0.1603],
         ...,
         [ 0.1852,  0.1852,  0.1852,  ...,  0.1852,  0.1852,  0.1852],
         [ 0.1420,  0.1420,  0.1420,  ...,  0.1420,  0.1420,  0.1420],
         [ 0.0394,  0.0394,  0.0394,  ...,  0.0394,  0.0394,  0.0394]]],
       grad_fn=<CatBackward0>)